In [90]:
# based on Create_PMA_ontology_JSON.py
# @author: Austin Hoag
# @date: 2020-09-17
# The purpose of this script is to create the ontology JSON file for the Princeton Mouse Atlas,
# just like what Allen provides for the AMBA.
# We will start from a CSV file containing the regions names, ids, and their parent:child relationships.
#

In [91]:
import json
import numpy as np
import pandas as pd

In [92]:
# Read in csv file into pandas dataframe
df_filename = '/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/labels/PRA_labels.csv'
#'/jukebox/LightSheetTransfer/atlas/Princeton_mouse_atlas_id_table_public.csv'
df_pra = pd.read_csv(df_filename)

In [93]:
df_pra=df_pra[['id','name','parent_id','parent_name']]

In [94]:
graph_order=0

In [95]:

def make_ontology_dict(dic):
    """
    ---PURPOSE---
    A recursive function that completes a
    dictionary of the same format as the allen.json file,
    using the pma_df_dict we just made.
    ---INPUT---
    dic:    A parent dictionary to which you want to add child dictionaries
    """
    # Find all children of the current ID
    parent_id = dic['id']
    child_ids = [v['id'] for v in pra_df_dict.values() if v['parent_id'] == parent_id]
    # add the correct graph_order value to this id
    global graph_order
    dic['graph_order'] = graph_order
    print('go in main body: {}'.format(graph_order))
    graph_order+=1 # increment so the next in line in the hierarchy has a higher graph_order
    """ Make list of child dicts to add to the entry in the ontology dictionary"""
    child_dicts = [pra_df_dict[ID] for ID in child_ids]
    dic['children']=child_dicts
    for child in child_dicts: 
        child_id = child['id']
        make_ontology_dict(child) # recursive call - child is a dictionary itself
        graph_order-=1
        print('child {} graph order {}'.format(child,graph_order))
    return

In [96]:
df_pra.columns = ['ids','name','parent_id','parent_name']

In [97]:
np.unique(df_pra['parent_id'])

array([  0,   2,   3,   4,   6,  10,  11,  12,  13,  30,  34,  35,  36,
        37,  40,  45,  51,  52,  58,  59,  60,  61,  64,  66,  74,  91,
        92, 104, 105, 111, 116, 117, 121, 122, 123, 125, 136, 150, 151,
       157, 158, 159, 189, 190, 192, 193, 200, 210, 212, 217, 232, 233,
       237, 238, 239, 240, 242, 244, 256, 266, 267, 285, 295, 297, 300,
       305, 306, 307, 313, 317, 318, 331, 356, 406, 407, 413, 417, 427,
       428, 440, 441, 445, 471, 505, 519, 534, 549, 551, 566, 567, 568,
       569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581,
       582, 584, 585, 586, 587, 588, 592, 595, 596, 597, 598, 599, 604,
       605, 606, 607, 608, 609, 610, 613, 614, 616, 617])

In [98]:
"""
Make a massive dictionary where keys are ids and values are dictionaries containing the
info we will need in the ontology JSON file
We will initialize each entry with an empty 'children' key
which we will fill using the recursive function below
"""
pra_df_dict = {}
for index, (ids,name,parent_id,parent_name) in df_pra.iterrows():
    if name != 'root': # we will handle the root dictionary separately below
        pra_df_dict[ids] = {'id':int(ids),'name':name,
                           'parent_id':int(parent_id),'graph_order':0,'children':[]}
# intialize the graph_order global variable that we will increment as we add the children
graph_order = 0 


# Initialize the final dictionary which will contain the nested structure with the root id 
ontology_dic = {'id':0,'common_name':'root','graph_order':0,'parent_structure_id':None,'children':[]}

# Call the recursive function, using the intialized root dictionary as the starting point
# The function fills out the rest of the dictionary
make_ontology_dict(dic=ontology_dic)

# Now write this to a JSON file
PRA_json_filename = './PRA_ontology.json'
with open(PRA_json_filename,'w') as outfile:
    json.dump(ontology_dic,outfile,indent=2) # indent is used to make formatting identical to allen.json
print("Wrote ontology dictionary to: ", PRA_json_filename)

go in main body: 0
go in main body: 1
go in main body: 2
go in main body: 3
child {'id': 171, 'name': 'ansoparamedian_fissure', 'parent_id': 580, 'graph_order': 3, 'children': []} graph order 3
go in main body: 3
child {'id': 172, 'name': 'fissure_intercrural', 'parent_id': 580, 'graph_order': 3, 'children': []} graph order 3
go in main body: 3
child {'id': 173, 'name': 'fissure_prepyramidal', 'parent_id': 580, 'graph_order': 3, 'children': []} graph order 3
go in main body: 3
child {'id': 170, 'name': 'interpeduncular_fossa', 'parent_id': 580, 'graph_order': 3, 'children': []} graph order 3
go in main body: 3
child {'id': 174, 'name': 'paraflocculus_sulcus', 'parent_id': 580, 'graph_order': 3, 'children': []} graph order 3
go in main body: 3
child {'id': 175, 'name': 'post_superior_fissure', 'parent_id': 580, 'graph_order': 3, 'children': []} graph order 3
go in main body: 3
child {'id': 176, 'name': 'posterolateral_fissure', 'parent_id': 580, 'graph_order': 3, 'children': []} graph o

In [99]:
ontology_dic

{'id': 0,
 'common_name': 'root',
 'graph_order': 0,
 'parent_structure_id': None,
 'children': [{'id': 4,
   'name': 'grooves',
   'parent_id': 0,
   'graph_order': 1,
   'children': [{'id': 580,
     'name': 'grooves_of_the_cerebellar_cortex',
     'parent_id': 4,
     'graph_order': 2,
     'children': [{'id': 171,
       'name': 'ansoparamedian_fissure',
       'parent_id': 580,
       'graph_order': 3,
       'children': []},
      {'id': 172,
       'name': 'fissure_intercrural',
       'parent_id': 580,
       'graph_order': 3,
       'children': []},
      {'id': 173,
       'name': 'fissure_prepyramidal',
       'parent_id': 580,
       'graph_order': 3,
       'children': []},
      {'id': 170,
       'name': 'interpeduncular_fossa',
       'parent_id': 580,
       'graph_order': 3,
       'children': []},
      {'id': 174,
       'name': 'paraflocculus_sulcus',
       'parent_id': 580,
       'graph_order': 3,
       'children': []},
      {'id': 175,
       'name': 'post_su